# Работа с реляционными БД

## Термины

* **БД** — база данных, то, где хранятся данные (вот так)
* **СУБД** — система управления базами данных — программное обеспечение для работы в с данными
* **Отношение** термин [*реляционной алгебры*](https://ru.wikipedia.org/wiki/%D0%A0%D0%B5%D0%BB%D1%8F%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D0%B0%D0%BB%D0%B3%D0%B5%D0%B1%D1%80%D0%B0) — аналог таблицы истинности логического отношения
* **Таблица** — набор данных в БД, соответствующий отношению в реляционной алгебре
* **Схема БД** — модель (формальное описание) данных, включающая в себя описания таблиц, дополнительных ограничений (например, ключей и ссылок между ними) и дополнительных избыточных структур данных (в частности, индексов).

## Аномалии редактирования данных

При неудачном проектировании схемы легко приести данные в некорректное состояние. Примеры:

* Очевидный пример: гражданин получает налоговое начисление за чужое имущество.
* Неочевидный (и спорный) пример: студент получает оценку по дисциплине, не входящей в его/её программу.

## Целостность и согласованность

* Целостность — соответствие данных в базе её модели (аналогично соответствия набора объектов их типам и типам ссылок в статически типизированных языках). Обычно обеспечивается СУБД.
* Согласованность — выполнение более сильных ограничений на корректность, не описанных в модели (например, что суммы на входе и выходе денежной транзакции одинаковые). Обеспечивается бизнес-логикой, часто опираясь на поддержку СУБД транзакций.


## Структура и нормализация схемы данных

1. 1NF — отношение состоит из различных кортежей одинаковой природы, элементы кортежей атомарны
2. 2NF — каждый неключевой атрибут функционально зависит от ключа
3. 3NF — нет транзитивных функциональных зависимостей
  * Boyce-Codd — атрибуты ключа не зависят от неключевых
4. 4NF и далее — ограничения, которые уже часто относят к согласованности и реже вносят в схему

...

См. Дейт К. Дж. Введение в системы баз данных

## Поехали уже =)

In [1]:
import sqlite3
import sqlalchemy

conn = sqlite3.connect("students.sqlite3")  # Как оригинально!
cursor = conn.cursor()

А дальше соединения с любой реляционной БД и (для любых типов реляционных СУБД) должны (желательно) удовлетворять [Python DB-API 2.0](https://www.python.org/dev/peps/pep-0249/).

In [ ]:
with open("11.drop_ddl.sql", 'r', encoding='utf-8') as f: drop_ddl = f.read()
with open("11.create_ddl.sql", 'r', encoding='utf-8') as f: create_ddl = f.read()
with open("11.insert_dml.sql", 'r', encoding='utf-8') as f: insert_dml = f.read()

In [ ]:
if True:
    cursor.executescript(drop_ddl)
    conn.commit()

In [ ]:
if True:
    cursor.executescript(create_ddl)
    conn.commit()

После этого *схема* БД станет такой:

![](img/students_scheme.png)

In [ ]:
cursor.executescript(insert_dml)
conn.commit()

In [ ]:
from IPython.display import HTML, display
import tabulate  # pip install tabulate

cursor.execute(
    '''select id, name from programs'''
)
display(HTML(tabulate.tabulate(cursor.fetchall(), tablefmt='html')))

cursor.execute(
    '''select programs.name, students.name, surname, patronymic
    from programs join students
    on programs.id = students.program_id''')

display(HTML(tabulate.tabulate(cursor.fetchall(), tablefmt='html')))

Чтобы не было как на самой первой иллюстрации, не только при редактировании, но и при чтении данных параметры запросов следует добавлять в запрос не при помощи конкатенации строк, а при помощи специальных средств драйвера. В зависимости от драйвера, СУБД и настроек, [синтаксис может отличаться](https://www.python.org/dev/peps/pep-0249/#paramstyle), в SQLite3 по умолчанию используется самый простой — qmark.

In [ ]:
name = "Василий"
surname = "Пупкин"
card = '199099'
cursor.execute(
    "insert into students(program_id, card, surname, name) values(1, ?, ?, ?)",
    (card, surname, name)
)
conn.commit()

In [ ]:
conn.close()

# А зачем такие сложности с параметрами?

Почему нельзя просто конкатенировать строки?

![Bobby Tables](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

Попробуйте на настоящей базе данных!

In [ ]:
name = input("Name?")
# try Robert'); DROP TABLE Students; --
sql = "insert into Students(name) values('" + name + "');"
print(sql)

# Теперь попробуем немного [ORM](https://en.wikipedia.org/wiki/Object-relational_mapping) — [SQLAlchemy](https://ru.wikibooks.org/wiki/SQLAlchemy)

In [ ]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

DeclBase = declarative_base()

In [ ]:
engine = sqlalchemy.create_engine('sqlite:///students.sqlite3', echo=False)  # echo=True для логгинга
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
class Program(DeclBase):
    __tablename__ = 'programs'
    id = Column(Integer, primary_key=True)
    name = Column(String)

    students = relationship("Student", backref="program")

    def __init__(self, name):
        self.name = name

class Student(DeclBase):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    card = Column(String)
    surname = Column(String)
    name = Column(String)
    patronymic = Column(String)
    
    program_id = Column(Integer, ForeignKey('programs.id'))
    
    def __init__(self, card, surname, name, patronymic, program):
        self.card = card
        self.surname = surname
        self.name = name
        self.patronymic = patronymic
        self.program = program

In [ ]:
se = Program("Программная инженерия")

st1 = Student("002002", "Иванов", "Пётр", "Сидорович", se)
st2 = Student("002003", "Петрова", "Исидора", "Ивановна", se)

session.add_all([se, st1, st2])
session.commit()

In [ ]:
import sys

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name)

In [ ]:
ph: Program = session.query(Program).filter_by(name="теоретическая физика")[0]
v: Student = session.query(Student).filter_by(name="Исидора")[0]

v.program = ph
session.commit()

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name)